# Scraping for data
### In this notebook recipes from the German Bundesverband der Verbraucherinitiative are scraped. This website was chosen, since it is supposed to have "healthy" recipes for different occasions and seasons.

In [4]:
%pip install -U gazpacho

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gazpacho: filename=gazpacho-1.1-py3-none-any.whl size=7489 sha256=265f7f9917539a2e811d7b04acf477383855dde7f1fb03551885efa1b1f10752
  Stored in directory: c:\users\carla\appdata\local\pip\cache\wheels\f4\e6\e3\d9a9b3f1dc045ee40c507cd630daa6cdd41b239633e92d5457
Successfully built gazpacho
Note: you may need to restart the kernel to use updated packages.


In [38]:
!pip install html5lib


   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   -------------- ------------------------ 41.0/112.2 kB 330.3 kB/s eta 0:00:01
   ---------------------------------- --- 102.4/112.2 kB 737.3 kB/s eta 0:00:01
   -------------------------------------- 112.2/112.2 kB 592.6 kB/s eta 0:00:00


### Importing most important libraries

In [1]:
import re
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup as bs
from gazpacho import Soup
import numpy as np

### Definiting the scraping function

In [3]:
import requests
from bs4 import BeautifulSoup as bs
from bs4 import Tag
import html5lib

all_ingredients = []
all_instructions = []
all_names = []

def scrape_me(url):
    response = requests.get(url)
    soup = bs(response.content, 'html5lib')
    
    div = soup.find("div", {"class": "mod-article__text mod-text"})
    article = soup.find("article", {"class": "mod-article"})
    
    if div:
        # Extract the ingredients and instructions within this div
        ingredients_ul = div.find("ul")
        instructions_ol = div.find("ol")

        if ingredients_ul and instructions_ol:
            ingredients_html = ingredients_ul.decode_contents()
            instructions_html = instructions_ol.decode_contents()

            # Extract the recipe name from the h1 tag
            if article:
                recipe_name = article.find("h1", {"class": "mod-article__headline"}).text.strip()
                
                # Parse the extracted HTML with BeautifulSoup for easier text extraction
                ingredients_soup = bs(ingredients_html, 'html5lib')
                instructions_soup = bs(instructions_html, 'html5lib')

                # Function to handle extraction of text from potentially malformed lists
                def extract_list_items(soup):
                    items = []
                    for li in soup.find_all("li"):
                        if isinstance(li, Tag):
                            items.append(li.get_text(strip=True))
                    return items

                # Extract the text and store in lists
                ingredients = extract_list_items(ingredients_soup)
                instructions = extract_list_items(instructions_soup)
                
                all_ingredients.append(ingredients)
                all_instructions.append(instructions)
                all_names.append(recipe_name)
                
                print(len(all_ingredients), len(all_instructions), len(all_names))
            else:
                print("Article not found")
        else:
            print("Ingredients or Instructions not found")
    else:
        print("Div not found")


### Scraping the website
### Thankfully the website was using a numbering system in their url, which made it possible to just loop through it

In [8]:
all_ingredients = []
all_instructions = []
all_names = []
i = 2

for i in range(2,332):
    print(i)
    scrape_me(f"https://verbraucher.org/informieren/rezept-datenbank?rezept=0{i}")

2
1 1 1
3
2 2 2
4
3 3 3
5
4 4 4
6
5 5 5
7
6 6 6
8
7 7 7
9
8 8 8
10
9 9 9
11
10 10 10
12
11 11 11
13
12 12 12
14
13 13 13
15
14 14 14
16
15 15 15
17
16 16 16
18
17 17 17
19
18 18 18
20
19 19 19
21
20 20 20
22
21 21 21
23
22 22 22
24
23 23 23
25
24 24 24
26
25 25 25
27
26 26 26
28
27 27 27
29
28 28 28
30
29 29 29
31
30 30 30
32
31 31 31
33
32 32 32
34
33 33 33
35
34 34 34
36
35 35 35
37
36 36 36
38
37 37 37
39
38 38 38
40
39 39 39
41
40 40 40
42
41 41 41
43
42 42 42
44
43 43 43
45
44 44 44
46
45 45 45
47
46 46 46
48
47 47 47
49
48 48 48
50
49 49 49
51
50 50 50
52
51 51 51
53
52 52 52
54
53 53 53
55
54 54 54
56
55 55 55
57
56 56 56
58
57 57 57
59
58 58 58
60
59 59 59
61
60 60 60
62
61 61 61
63
62 62 62
64
Div not found
65
63 63 63
66
64 64 64
67
Div not found
68
65 65 65
69
66 66 66
70
67 67 67
71
68 68 68
72
69 69 69
73
Div not found
74
70 70 70
75
71 71 71
76
72 72 72
77
73 73 73
78
74 74 74
79
75 75 75
80
76 76 76
81
77 77 77
82
78 78 78
83
79 79 79
84
80 80 80
85
81 81 81
86
82 82 82


### Saving the data

In [10]:
for e in range(0, len(all_names)):
    data = {"Name": all_names,
            "Ingredients": all_ingredients,
            "Instructions": all_instructions}
df = pd.DataFrame(data)
df
df.to_csv("data/rezepte.csv", index=False)